In [1]:
# coding:utf-8
from spider import refresh_data, get_webpage_content

url = "https://www.jisilu.cn/web/data/cb/list"
driver = get_webpage_content(url)
data = refresh_data(driver)

2025-09-10 17:10:30.742 - INFO - spider.py - 48 - 进入登陆界面
2025-09-10 17:10:30.931 - INFO - spider.py - 84 - 登陆成功
2025-09-10 17:10:31.486 - INFO - spider.py - 103 - 刷新成功
2025-09-10 17:10:31.647 - INFO - spider.py - 115 - 表头采集完成
2025-09-10 17:10:37.153 - INFO - spider.py - 131 - 表体采集完成


In [2]:
data

,行号,操作,代码,转债名称,现价,涨跌幅,正股代码,正股名称,正股价,正股涨跌,...,强赎触发价,转债流通市值占比,基金持仓,到期时间,剩余年限,剩余规模(亿元),成交额(万元),换手率,到期税前收益,回售税前收益
0,1,,123254,亿纬转债,147.566,-3.68%,300014,亿纬锂能,70.15,-2.88%,...,66.157,3.80%,1.56%,31-03-24,5.54年,50.000,114716.24,15.56%,-4.24%,-903.7%
1,2,,123215,铭利转债,139.799,1.04%,301268,铭利达,23.95,3.72%,...,24.960,19.80%,25.43%,29-08-03,3.90年,8.555,11195.16,9.41%,-3.39%,-13.11%
2,3,,118031,天23转债,123.957,-1.49%,688599,天合光能,17.10,-2.79%,...,20.800,23.80%,32.54%,29-02-13,3.43年,88.645,28363.70,2.58%,-1.13%,-10.36%
3,4,,113685,升24转债,135.186,-1.84%,603305,旭升集团,15.61,-0.95%,...,16.263,17.10%,41.01%,30-06-14,4.76年,25.409,34834.02,10.02%,-3.16%,-8.62%
4,5,,123159,崧盛转债,150.040,-3.33%,301002,崧盛股份,35.84,-1.86%,...,31.655,11.20%,18.53%,28-09-27,3.05年,2.943,54579.59,123.86%,-7.60%,-24.52%
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
431,432,,113695,华辰转债,144.900,0.10%,603097,江苏华辰,22.31,0.13%,...,30.589,12.90%,0.00%,31-06-19,5.78年,4.600,1201.03,1.81%,-3.41%,-8.05%
432,433,,118020,芳源转债,121.273,-0.36%,688148,芳源股份,6.35,-3.35%,...,24.219,19.80%,17.42%,28-09-23,3.04年,6.417,2990.39,3.85%,1.64%,-10.96%
433,434,,123034,通光转债,196.500,0.64%,300265,通光线缆,9.19,-0.65%,...,10.244,1.80%,-,25-11-04,55天,0.771,89417.17,573.16%,-287.3%,-
434,435,,113575,东时转债,183.270,-4.79%,603377,ST东时,3.75,-2.60%,...,15.795,3.60%,-,26-04-09,0.58年,0.976,62039.38,345.01%,-71.39%,-63.53%


In [5]:
type(data)

pandas.core.frame.DataFrame

In [ ]:
import polars as pl
import pandas as pd
# 计算溢价率
def fixed_fair_value(fair_value, bond_par_value, convert_value, bond_trans_value):
    """
    return: 债券交易价格得到的溢价率
    """
    # B_T = B_P * S_F / S_C
    convert = (fair_value * bond_par_value / convert_value)
    return 100*(bond_trans_value - convert) / convert

# 计算溢价率和表中溢价率的差
def difference(fair_value, convert_value, bond_trans_value):
    return fixed_fair_value(fair_value, 100, convert_value, bond_trans_value)

# 评价差
def get_target(x):
    if (x >= 0.1 and x< 1) or (x <= -0.1 and x > -1):
        return '差异>=0.1%且差异<=1%'
    elif (x >= 1 and x < 10) or (x <= -1 and x > -10):
        return '差异>1%且差异<10%'
    elif (x >= 10 or x <= -10):
        return '差异>10%'
    else:
        return ''


# 计算涨跌幅差异
def rate_diff(stock_rate, bond_rate):
    if (stock_rate > 0 and bond_rate > 0):  # 债券的涨跌幅是大于0的，也就是说债券是往上发展的
        if (stock_rate - bond_rate >= 4.2) or (bond_rate - stock_rate >= 4.2):   # 正股比债股涨跌幅大
            return '债股差异大'
        return ''
    else:
        return ''


def calc_columns(df):
    if isinstance(df, pd.DataFrame):
        df = pl.from_pandas(df)

    df = df.with_columns([
        # 数据类型转换——直接转
        pl.col('现价').cast(pl.Float64),
        pl.col('正股价').cast(pl.Float64),
        pl.col('转股价').str.count_matches(r'\*').alias('下修次数'),

        # 数据类型转换——处理后转
        pl.col('正股涨跌').str.replace('%', '').cast(pl.Float64, strict=False)
        .alias('正股涨跌'),
        
        # 处理涨跌幅：去除%符号，空值处理为0
        pl.col('涨跌幅').str.replace('%', '').cast(pl.Float64, strict=False)
        .alias('涨跌幅'),
        
        pl.col('转股溢价率').str.replace('%', '').cast(pl.Float64, strict=False)
        .alias('转股溢价率'),  
              
        pl.col('转股价').str.replace_all(r'\*+', '').cast(pl.Float64).alias('转股价')
    ])
    
    # 新增列计算
    df = df.with_columns([
        pl.struct(['正股价', '转股价', '现价'])
        .map_elements(lambda x: difference(x['正股价'], x['转股价'], x['现价']), return_dtype=pl.Float64)
        .alias('计算溢价率')
    ]).with_columns([
        (pl.col('计算溢价率') - pl.col('转股溢价率')).alias('差异')
    ]).with_columns([
        pl.struct(['正股涨跌', '涨跌幅'])
        .map_elements(lambda x: rate_diff(x['正股涨跌'], x['涨跌幅']), return_dtype=pl.Utf8)
        .alias('涨跌差异'),

        pl.col('差异').map_elements(get_target, return_dtype = pl.Utf8).alias('标签')
    ])

    # 筛选过滤条件
    df_filter = df.filter(
        (pl.col('涨跌差异') != '') & (pl.col('转股溢价率') <= 50)
    )

    result_col = [
        '转债名称', '代码', '正股名称', '正股代码', '现价', 
        '正股价', '涨跌幅', '正股涨跌', '涨跌差异', '转股价', 
        '转股溢价率', '差异', '计算溢价率', '标签'
    ]
    # logger.info("字段处理完成")
    return df_filter[result_col]


In [24]:
a = calc_columns(data)
a

AttributeError: 'ExprStringNameSpace' object has no attribute 'strip'

In [5]:
for i in range(len(bodies)):
    print(len(bodies[i]), bodies[i])

30 ['1', '\ue61e', '123254', '亿纬转债', '147.010', '-4.04%', '300014', '亿纬锂能', '70.37', '-2.58%', '3.69', '50.89', '138.28', '6.31%', '153.32', '103.23', 'AA+', '85.36', '59.80%', '35.623', '66.157', '3.80%', '1.56%', '31-03-24', '5.54年', '50.000', '101798.15', '13.80%', '-4.17%', '-896.5%']
30 ['2', '\ue61e', '123215', '铭利转债', '139.517', '0.83%', '301268', '铭利达', '23.84', '3.25%', '5.23', '19.20*', '124.17', '12.36%', '151.88', '109.30', 'AA-', '67.84', '63.84%', '13.440', '24.960', '19.90%', '25.43%', '29-08-03', '3.90年', '8.555', '9952.33', '8.37%', '-3.34%', '-13.03%']
30 ['3', '\ue61e', '118031', '天23转债', '123.992', '-1.46%', '688599', '天合光能', '17.12', '-2.67%', '1.60', '16.00**', '107.00', '15.88%', '139.87', '111.02', 'AA', '45.97', '55.09%', '11.200', '20.800', '23.80%', '32.54%', '29-02-13', '3.43年', '88.645', '25286.94', '2.30%', '-1.14%', '-10.38%']
30 ['4', '\ue61e', '123258', '胜蓝转02', '130.000', '停牌', '300843', '胜蓝股份', '55.84', '3.22%', '5.81', '54.56', '102.35', '27.02%', '1

In [4]:
len(bodies)

436